In [1]:
from pathlib import Path
import json
import pandas as pd
from ollama import Client
from tqdm import tqdm
import string

In [6]:
df = pd.read_csv('experiments/paraphrasing/task4report.csv')
df.head()

,Model,Dataset,Language,Task,Rouge 1,Rouge 2,Rouge L,Rouge Lsum,Exact Match,F1,METEOR
0,gemma2,DisflQA,en,CORR,83.28,71.99,80.66,80.67,21.77,83.10,89.08
1,llama3.1,DisflQA,en,CORR,78.94,65.33,75.80,75.80,11.45,78.46,85.69
2,mistral,DisflQA,en,CORR,78.21,63.10,75.24,75.24,15.21,77.77,81.38
3,gemma2,DISCO,en,CORR,92.40,85.56,91.63,91.64,64.44,92.59,91.55
4,llama3.1,DISCO,en,CORR,91.20,84.57,90.58,90.58,64.01,91.43,91.07


In [8]:
# remove columns dataset and language
df = df.drop(columns=['Dataset', 'Language'])

In [9]:
# aggregate by model and task, averaging over the runs
df = df.groupby(['Model', 'Task']).mean().reset_index()

In [13]:
df = df.round(2)

In [14]:
df.to_csv('experiments/paraphrasing/task4report_aggregated.csv', index=False)

In [ ]:
client = Client(host='http://localhost:19290')
client.list()

In [ ]:
df = pd.read_parquet("datasets/disco/disco_en.parquet")
df.head()

In [ ]:
system_prompt = "You are a system to support the analysis of large amounts of text. You will assist the user by rephrasing the provided texts. You will always answer in the required format and use no other formatting than expected by the user!"

In [ ]:
user_prompt = """
I have a noisy, disfluent text. I need you to remove all disfluencies from the text below. Keep the text as close to the original as possible, but make sure it is fluent to read.

{}

Respond in the following format:
Fluent text: <the corrected text>

e.g.
Fluent text: This picture looks great.

Remember, you MUST keep to the original text as much as possible, do not generate new content!
"""

In [ ]:
def parse_response(response):
    result = ""

    for line in response.strip().split("\n"):
        if not line.strip():
            continue
        if ":" not in line:
            continue

        splitted_line = line.split(":")
        if len(splitted_line) != 2:
            continue

        result = splitted_line[1].strip()

    return result

In [ ]:
messages = []
golds = []
preds = []
for idx, sample in tqdm(df[:10].iterrows(), desc="Evaluating"):
    response = client.chat(model='gemma2', messages=[
        {
            'role': 'system',
            'content': system_prompt.strip(),
        },
        {
            'role': 'user',
            'content': user_prompt.format(sample["Disfluent Sentence"]).strip(),
        },
    ])
    message = response["message"]["content"]
    pred = parse_response(message)

    messages.append(message)
    preds.append(pred)
    golds.append(sample["Fluent Sentence"])

    print(sample["Disfluent Sentence"])
    print("Gold text:", sample["Fluent Sentence"])
    print(message)
    print()

In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
preds

In [ ]:
golds

In [ ]:
from typing import List


def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))

def to_squad(text: List[str], is_gold) -> List[str]:
    result = []
    for idx, sentence in enumerate(text):
        if is_gold:
            result.append({'answers': {"answer_start": [0], "text": [sentence]}, "id": str(idx)})
        else:
            result.append({'prediction_text': sentence, 'id': str(idx), 'no_answer_probability': 0.0})

    return result


In [ ]:
rouge.compute(predictions=preds, references=golds)

In [ ]:
rouge.compute(predictions=[remove_punctuation(x) for x in preds], references=[remove_punctuation(x) for x in golds])

In [ ]:
preds = [remove_punctuation(x) for x in preds]
golds = [remove_punctuation(x) for x in golds]

preds_transformed = to_squad(preds, False)
golds_transformed = to_squad(golds, True)

In [ ]:
squad_v2_metric = evaluate.load("squad_v2")
print(squad_v2_metric.compute(references=golds_transformed, predictions=preds_transformed))

# Ergebnisse angucken

In [1]:
import pandas as pd


df = pd.read_parquet("experiments/paraphrasing/paraphrasing_mlsum.parquet")

In [2]:
df["Prediction"].tolist()

['Die Kapitänin der Sea Watch 3, Carola Rackete, brachte trotz Blockades durch die italienische Regierung 40 Migranten nach Lampedusa. Sie argumentierte mit einem Notstand an Bord und droht nun wegen Beihilfe zur illegalen Einwanderung mehrere Anklagen. International gibt es große Solidarität mit Rackete.',
 'Carola Rackete rettete Flüchtlinge aus Seenot und brachte sie nach Lampedusa, da kein anderes Land bereit war, sie aufzunehmen.  Sie wird nun wegen ihres Handelns angeklagt, während Italien unter Salvini die Hafenblockade beibehält und Flüchtlingshilfen kriminalisiert.',
 'Der sächsische Ministerpräsident Kretschmer hielt eine Rede vor der CDU-Delegation, in der er sich gegen Rechtsextremismus aussprach und ein starkes Programm für Sachsen vorstellte. Er will das Land sicherer machen, die Wirtschaft stärken und junge Menschen anziehen, während er die AfD als Gegner bezeichnet.',
 'Trotz der Wiederaufnahme der Handelsgespräche zwischen USA und China besteht die Gefahr eines erneute

In [3]:
df["Gold"].tolist()

['Carola Rackete ist auf Lampedusa festgenommen worden. Außenminister Heiko Maas und sein Kollege Jean Asselborn solidarisierten sich mit der Kapitänin der "Sea Watch 3". Andere kritisierten ihr Anlegemanöver.',
 'Carola Rackete hat gemacht, was selbstverständlich sein sollte: Menschen aus Seenot retten. Dass ihr dafür nun Haft droht, ist auch für Europa ein Armutszeugnis.',
 'Die sächsische CDU bereitet sich beim Landesparteitag auf einen harten Wahlkampf vor. Ministerpräsident Kretschmer macht klar, wo der Gegner steht - und hält ein Plädoyer gegen Rechtsextremismus.',
 'Dass die USA und China ihre Handelsgespräche wiederaufnehmen, ist erst einmal eine gute Nachricht. Solange beide Seiten Handel aber als eine Art Ausscheidungskampf begreifen, ist ein Kompromiss unmöglich.',
 'Die Kosten für das Wohnen sind nicht so extrem gestiegen, wie viele behaupten.',
 'Unbekannte haben dem sächsischen SPD-Vorsitzenden Martin Dulig einen Sturmgewehr-Nachbau zugeschickt - an seine Privatadresse. M

# ML SUM

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mlsum", "de")

In [ ]:
dataset

In [ ]:
dataset["test"]

In [ ]:
dataset["test"][0]


# CNN DM

In [ ]:
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [ ]:
dataset

In [ ]:
dataset["test"]

In [ ]:
dataset["train"]["highlights"][4]

In [ ]:
dataset["test"]["article"][0]